In [ ]:
# default_exp core

In [ ]:
#hide
from nbdev.showdoc import show_doc

# Core

> Core tools ...  TODO

In [ ]:
#export
POST_STATUS_MAP={50:'default',40:'in-progress',30:'done',20:'deleted'}

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 40a_service_db.ipynb.
Converted 40b_service_filesystem.ipynb.
Converted 50_web_app.ipynb.
Converted 50b_web_auth.ipynb.
Converted 50c_web_blog.ipynb.
Converted index.ipynb.
